In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal
from gdalconst import *
import ee
import geemap
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
import ee
import geemap

Map = geemap.Map()
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AfDhmrg7mI6pk1B5tP1FtinRjeSIJkJgm2YNC5UKH4lpAzSGBjhe57YKtgg

Successfully saved authorization token.


In [3]:
SR_ls8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
SR_ls7 = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR")
aoi = ee.FeatureCollection("TIGER/2018/States").filterMetadata('NAME', 'equals', 'Iowa')
aoi = aoi.geometry()

In [4]:

def rename(image):
    if image.get('SATELLITE') == 'LANDSAT_7':
        image = image.select(['B4', 'B5']).rename(['NIR', 'SWIR'])
    else:
        image = image.select(['B5', 'B6']).rename(['NIR', 'SWIR']);
    return image

def cloud_mask(image):
#     3 nd 5 are cloud shadow and cloud, respectively.
     cloudShadowBitMask = ee.Number(2).pow(3).int();
     cloudBitMask = ee.Number(2).pow(5).int();
#   Get the pixel QA band.
     qa = image.select('pixel_qa');
#   Both flags should be set to zero, indicating clear conditions.
     mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cloudShadowBitMask).eq(0));
#   Return the masked image, scaled to [0, 1].
     return image.addBands(mask.rename('mask'))

def apply_mask(image):
    return image.updateMask(image.select('mask'))

In [20]:
def cloudfree_sr7(date):
    aoi = ee.FeatureCollection("TIGER/2018/States").filterMetadata('NAME', 'equals', 'Iowa')
    aoi = aoi.geometry().bounds()
    increase = ee.Number(date)
    start = ee.Date('2019-06-01').advance(increase.multiply(16), 'day')
    end = ee.Date('2019-06-17').advance(increase.multiply(16), 'day')
#     SR = SR_ls8.filterBounds(aoi).merge(SR_ls7.filterBounds(aoi)).filterDate(start, end).map(cloud_mask)
    SR = SR_ls7.filterBounds(aoi).filterDate(start, end).map(cloud_mask)
    SR_mask = SR.map(apply_mask)
    return SR_mask.mosaic()  ##revised by ZJ
def cloudfree_sr8(date):
    aoi = ee.FeatureCollection("TIGER/2018/States").filterMetadata('NAME', 'equals', 'Iowa')
    aoi = aoi.geometry().bounds()
    increase = ee.Number(date)
    start = ee.Date('2019-06-01').advance(increase.multiply(16), 'day')
    end = ee.Date('2019-06-17').advance(increase.multiply(16), 'day')
#     SR = SR_ls8.filterBounds(aoi).merge(SR_ls7.filterBounds(aoi)).filterDate(start, end).map(cloud_mask)
    SR = SR_ls8.filterBounds(aoi).filterDate(start, end).map(cloud_mask)
    SR_mask = SR.map(apply_mask)
    return SR_mask.mosaic()  ##revised by ZJ

Functions for mosaic sentienl-2 imagery

In [21]:
CDL = ee.ImageCollection("USDA/NASS/CDL").filterDate('2015-01-01', '2015-12-31')

In [22]:
for i in range(0, 8):
    intervals = ee.List.sequence(i, i)
    countCol_sr7 = intervals.map(cloudfree_sr7)
    countCol_sr8 = intervals.map(cloudfree_sr8)
#     band = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']
    band_ls7 = ['B4', 'B5']
    band_ls8 = ['B5', 'B6']
    geemap.ee_export_image_to_drive(ee.Image(countCol_sr7.get(0)).select(band_ls7), \
                         description="IW_2019_ls7" + str(i), \
                         folder='GEE_landsat_2019', \
                         max_pixels=1e13, \
                         region=aoi, \
                         crs='EPSG:4326', \
                         scale=30)
    geemap.ee_export_image_to_drive(ee.Image(countCol_sr8.get(0)).select(band_ls8), \
                     description="IW_2019_ls8" + str(i), \
                     folder='GEE_landsat_2019', \
                     max_pixels=1e13, \
                     region=aoi, \
                     crs='EPSG:4326', \
                     scale=30)

Exporting IW_2019_ls70 ...
Exporting IW_2019_ls80 ...
Exporting IW_2019_ls71 ...
Exporting IW_2019_ls81 ...
Exporting IW_2019_ls72 ...
Exporting IW_2019_ls82 ...
Exporting IW_2019_ls73 ...
Exporting IW_2019_ls83 ...
Exporting IW_2019_ls74 ...
Exporting IW_2019_ls84 ...
Exporting IW_2019_ls75 ...
Exporting IW_2019_ls85 ...
Exporting IW_2019_ls76 ...
Exporting IW_2019_ls86 ...
Exporting IW_2019_ls77 ...
Exporting IW_2019_ls87 ...
